In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
#tf.compat.v1.set_random_seed(1)
#sess = tf.Session(graph=tf.get_default_graph())
#K.set_session(sess)
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import pickle
from video_process_utils import *
import collections

C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
target_col = "GDI"

In [4]:
#assign train/validation/test ids
alldata_processed =\
    pd.read_csv("./data/processed/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed.groupby(['videoid','side']).head(1)
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [5]:
datasplit_df = pd.read_csv('./data/processed/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [6]:
with open('./data/processed/all_processed_video_segments.pickle', 'rb') as handle:
    processed_video_segments = pickle.load(handle)

In [7]:
x_columns = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,
                  2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,50,51,52,53,54,55,56]

target_dict = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    target_dict[row['videoid']] = row[target_col]

In [8]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)[:,:,x_columns]

In [9]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)[:,:,x_columns]

In [10]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)[:,:,x_columns]

In [11]:
y = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in all_ids])

In [12]:
y_train = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in train_ids])

In [13]:
y_validation = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids])

In [14]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [15]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [16]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)

In [17]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)

y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights]))

In [18]:
vid_length = 124

In [19]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        #multiply by len(weights) to make the magnitude invariant to number of components in target
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences of course, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]
normal_weights = [1.0,0]


#normalize weights to sum to 1 to prevent affecting loss function
weights = weights/np.sum(weights)
normal_weights = normal_weights/np.sum(normal_weights)

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#monitor our actual objective
mse_metric = w_mse(target_range**2*normal_weights)

hyper_str = "params_"
for param in [initial_lrate,epochs_drop,drop_factor,dropout_amount,conv_dim,last_layer_dim,filter_length,l2_lambda]:
    hyper_str = hyper_str + str(param) + "_"

K.clear_session()
#K.set_session(sess)

model = Sequential()
model.add(Conv1D(conv_dim,filter_length, input_dim=X_train.shape[2],input_length=vid_length,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(conv_dim,filter_length,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='linear'))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 8, padding="same", input_shape=(124, 23))`


In [20]:
checkpoint_folder = "./data/checkpoints/cnn_single_checkpoints_%s" % (target_col)

In [21]:
from keras.callbacks import LearningRateScheduler

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN

train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss_1:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)

    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_2', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=256, epochs=n_epochs,shuffle=True)

Train on 20426 samples, validate on 2469 samples
Epoch 1/100
20426/20426 [==============================] - 44s 2ms/step - loss: 0.2231 - loss_1: 1081.4768 - val_loss: 0.0848 - val_loss_1: 264.3009

Epoch 00001: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-01-264.3009.hdf5
Epoch 2/100
20426/20426 [==============================] - 33s 2ms/step - loss: 0.0648 - loss_1: 160.5145 - val_loss: 0.0586 - val_loss_1: 145.9953

Epoch 00002: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-02-145.9953.hdf5
Epoch 3/100
20426/20426 [==============================] - 33s 2ms/step - loss: 0.0493 - loss_1: 120.2455 - val_loss: 0.0454 - val_loss_1: 126.3641

Epoch 00003: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-03-126.3641.hdf5
Epoch 4/100
20426/20426 [==============================] - 39s 2ms/step - loss: 0.0389 - loss_1: 109.1099 - val_loss: 0.0342 - val_loss_1: 105.6182

Epoch 00004: saving model to ./data/checkpoints/cnn_si

20426/20426 [==============================] - 32s 2ms/step - loss: 0.0118 - loss_1: 66.3044 - val_loss: 0.0127 - val_loss_1: 71.6558

Epoch 00034: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-34-71.6558.hdf5
Epoch 35/100
20426/20426 [==============================] - 37s 2ms/step - loss: 0.0117 - loss_1: 65.8459 - val_loss: 0.0123 - val_loss_1: 69.6198

Epoch 00035: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-35-69.6198.hdf5
Epoch 36/100
20426/20426 [==============================] - 32s 2ms/step - loss: 0.0117 - loss_1: 65.6246 - val_loss: 0.0123 - val_loss_1: 69.2663

Epoch 00036: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-36-69.2663.hdf5
Epoch 37/100
20426/20426 [==============================] - 39s 2ms/step - loss: 0.0116 - loss_1: 65.4920 - val_loss: 0.0127 - val_loss_1: 71.9610

Epoch 00037: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-37-71.9610.hdf5
Epoch 38/100
20426/20426 

20426/20426 [==============================] - 52s 3ms/step - loss: 0.0097 - loss_1: 54.2514 - val_loss: 0.0120 - val_loss_1: 68.2285

Epoch 00067: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-67-68.2285.hdf5
Epoch 68/100
20426/20426 [==============================] - 48s 2ms/step - loss: 0.0097 - loss_1: 54.3117 - val_loss: 0.0121 - val_loss_1: 68.6797

Epoch 00068: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-68-68.6797.hdf5
Epoch 69/100
20426/20426 [==============================] - 42s 2ms/step - loss: 0.0096 - loss_1: 54.1658 - val_loss: 0.0117 - val_loss_1: 66.2706

Epoch 00069: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-69-66.2706.hdf5
Epoch 70/100
20426/20426 [==============================] - 38s 2ms/step - loss: 0.0094 - loss_1: 52.9440 - val_loss: 0.0113 - val_loss_1: 63.9053

Epoch 00070: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-70-63.9053.hdf5
Epoch 71/100
20426/20426 

20426/20426 [==============================] - 35s 2ms/step - loss: 0.0083 - loss_1: 46.7335 - val_loss: 0.0114 - val_loss_1: 65.1545

Epoch 00100: saving model to ./data/checkpoints/cnn_single_checkpoints_GDI/weights-100-65.1545.hdf5


In [22]:
import statsmodels.api as sm

In [23]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [24]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename'].apply(lambda x: int(x.split('-')[1]))
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [25]:
def predict_and_aggregate_singlevar(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    df["videoid"] = np.array(videoids)
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [26]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
predictions_df[target_col] = y
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [27]:
for i in range(0,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

weights-01-264.3009.hdf5
weights-02-145.9953.hdf5
weights-03-126.3641.hdf5
weights-04-105.6182.hdf5
weights-05-243.6832.hdf5
weights-06-104.0433.hdf5
weights-07-142.6263.hdf5
weights-08-87.8517.hdf5
weights-09-80.3443.hdf5
weights-10-80.3093.hdf5
weights-11-83.6928.hdf5
weights-12-81.3895.hdf5
weights-13-85.3664.hdf5
weights-14-86.8280.hdf5
weights-15-156.5216.hdf5
weights-16-77.5592.hdf5
weights-17-105.6925.hdf5
weights-18-87.0401.hdf5
weights-19-74.2190.hdf5
weights-20-79.0529.hdf5
weights-21-93.1087.hdf5
weights-22-73.7484.hdf5
weights-23-82.5394.hdf5
weights-24-64.5086.hdf5
weights-25-64.5004.hdf5
weights-26-69.2371.hdf5
weights-27-81.7981.hdf5
weights-28-73.3958.hdf5
weights-29-82.7430.hdf5
weights-30-68.6514.hdf5
weights-31-89.0309.hdf5
weights-32-64.3107.hdf5
weights-33-71.1331.hdf5
weights-34-71.6558.hdf5
weights-35-69.6198.hdf5
weights-36-69.2663.hdf5
weights-37-71.9610.hdf5
weights-38-66.9985.hdf5
weights-39-69.8560.hdf5
weights-40-69.4553.hdf5
weights-41-74.6373.hdf5
weights

In [28]:
predictions_df.groupby(['videoid','dataset'],as_index=False).mean().to_csv("./data/predictions/cnn_%s_predictions_all_epochs.csv" % (target_col),index=False)